In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

# authentication to access secured data
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
gis = GIS('pro')
target_sr = arcpy.SpatialReference(2223)
db = arcpy.env.workspace = r".\data\sc_wezo_pavement_final.gdb"
arcpy.Describe(db)

custom functions.

In [ ]:
# save hosted feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

# project in_data to match the target spatial reference
def project(in_data, target_sr, out_name):


    in_sr = arcpy.Describe(in_data).spatialReference

    tr = arcpy.ListTransformations (in_sr, target_sr)
    datum_conversion = ''
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

Export sc routes layer. 

In [ ]:
route_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Segmented_Routes/FeatureServer/0'

save_fl(db, route_url, 'sc_routes')

Export the wezo pavement. 

In [ ]:
in_data = r'.\data\nira_combined_tiles.gpkg\main.NIRA Roughness' 
arcpy.ExportFeatures_conversion(in_data, 'pavement_raw')

Delete identical source segments. (1.4 mil -> )

In [ ]:
arcpy.management.DeleteIdentical(
    in_dataset="pavement_raw",
    fields="Shape;Long_term_Roughness;Long_term_Roughness_Uncertainty;Current_Roughness;Current_Roughness_Variance;Latest_measurement_update;FunctionalRoadClass;NvdbRoadClass;Shape_Length",
    xy_tolerance=None,
    z_tolerance=0
)

Unsplit the source, if functional class and roughness values are the same. 

In [ ]:
arcpy.management.UnsplitLine(
    in_features="pavement_raw",
    out_feature_class="pavement_raw_UnsplitLine",
    dissolve_field="Long_term_Roughness;Long_term_Roughness_Uncertainty;FunctionalRoadClass",
    statistics_fields=None,
    concatenation_separator=""
)

In [ ]:
# add new fields to the sc routes layer
# ['long_term_roughness_sum', 'DOUBLE', 'Long Term Roughness (Sum)'], 
arcpy.management.AddFields(
    'sc_routes', 
    [['average_iri_uncertainty', 'DOUBLE', 'Long Term Roughness Uncertainty (Wt.Avg.)'],
     ['average_iri', 'DOUBLE', 'Long Term Roughness (Wt.Avg.)'],
     ['source_seg_sum_mi', 'DOUBLE','Source Segment Length(Mi)']]
     )

Process for each functinal class pair. [(suncloud_fc, wezo_fc)]

In [ ]:
import pandas
import numpy as np

buffer_dist = '75 Feet'

# sc, wezo functional class match pairs
pairs = [(1,1), (1,2),
(2,2), (2, 1), (2,3), (2,4),
(3, 3), (3, 2), (3, 1), (3, 4),
(4, 4), (4,3), (4,1),
(5, 4), (5,3),
(6, 4),
(7, 4)]



for route_fc, wezo_fc in pairs:

    buffer = 'sc_routes_{0}{1}_Buffer'.format(route_fc,wezo_fc)
    intersect = "sc_routes_{0}{1}_Buffer_Intersect".format(route_fc,wezo_fc)

    # select 
    query = "functional_classification = '{}' and average_iri is null and average_iri_uncertainty is null".format(route_fc)
    print(query)
    selection = arcpy.SelectLayerByAttribute_management("sc_routes", 'NEW_SELECTION', query)

    arcpy.analysis.Buffer(
        in_features=selection,
        out_feature_class= buffer,
        buffer_distance_or_field=buffer_dist,
        line_side="FULL",
        line_end_type="FLAT",
        dissolve_option="NONE",
        dissolve_field=None,
        method="PLANAR"
    )
    # source FunctionalRoadClass = 1
    wezo_query = 'FunctionalRoadClass = {}'.format(wezo_fc)
    print(wezo_query)
    source_sel = arcpy.SelectLayerByAttribute_management("pavement_raw_UnsplitLine", 'NEW_SELECTION', wezo_query)
    intersect_tb = arcpy.analysis.Intersect(
        in_features=[ buffer, source_sel],
        out_feature_class=intersect,
        join_attributes="ALL",
        cluster_tolerance=None,
        output_type="INPUT")


    # intersect_tb = intersect
    fields = ['ORIG_FID','Long_term_Roughness', 'Shape_Length','Long_term_Roughness_Uncertainty']

    value_pairs = [row for row in arcpy.da.SearchCursor(intersect_tb, fields)]
    # print(value_pairs)
    if(len(value_pairs)>0):
    
        df = pandas.DataFrame(value_pairs)
        df.columns = ['orig_fid', 'roughness', 'shape_length', 'uncertainty']
        def f(x):
            d = {}
            d['roughness'] = sum(x['roughness'] * x['shape_length']) / sum(x['shape_length'])*63.36

            d['uncertainty'] = sum(x['uncertainty'] * x['shape_length']) / sum(x['shape_length'])
            d['length_sum_mi'] = sum(x['shape_length']*0.000189394)

            return pandas.Series(d, index=['roughness', 'uncertainty', 'length_sum_mi'])


        # export dataframe to csv
        df2 = df.groupby(['orig_fid']).apply(f)
        csv = r".\data\class_{0}{1}.csv".format(route_fc,wezo_fc)
        df2.to_csv(csv)

        # convert csv to esri table
        out_tb = "class_{0}{1}".format(route_fc,wezo_fc)
        arcpy.conversion.ExportTable(
            in_table=csv,
            out_table=out_tb
        )


        # join and calculate
        joined = arcpy.management.AddJoin(
            in_layer_or_view="sc_routes",
            in_field="OBJECTID",
            join_table=out_tb,
            join_field="orig_fid",
            join_type="KEEP_COMMON",
            index_join_fields="NO_INDEX_JOIN_FIELDS"
        )

        # calculate field
        arcpy.management.CalculateField(
            in_table=joined,
            field="sc_routes.average_iri",
            expression="!{}.{}!".format(out_tb,'roughness'),
            expression_type="PYTHON3",
        )


        arcpy.management.CalculateField(
            in_table=joined,
            field="sc_routes.average_iri_uncertainty",
            expression="!{}.{}!".format(out_tb,'uncertainty'),
            expression_type="PYTHON3",
        )

        arcpy.management.CalculateField(
            in_table=joined,
            field="sc_routes.source_seg_sum_mi",
            expression="!{}.{}!".format(out_tb,'length_sum_mi'),
            expression_type="PYTHON3",
        )


Decode the functional class

In [7]:
fc_dict = {
    1 : 'Interstate',
    2 : 'Other Freeways and Expressways',
    3 : 'Other Principal Arterial',
    4 : 'Minor Arterial',
    5 : 'Major Collector',
    6 : 'Minor Collector',
    7:  'Local'
}

arcpy.management.CalculateField(
    in_table="sc_routes",
    field="functional_classification",
    expression="convert(!functional_classification_code!)",
    expression_type="PYTHON3",
    code_block="""
def convert(code):
    return fc_dict.get(code)""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result '.\\data\\sc_wezo_pavement_final.gdb\\sc_routes'>